In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization

Using TensorFlow backend.


In [2]:
import numpy as np
import os

np.random.seed(42)

BASE_PATH = '../'
DATA_PATH = BASE_PATH + 'dataset'
MODELS_DIR = BASE_PATH + "models"

In [3]:
from scipy.io import arff
import pandas as pd

def load_pulsar_csv(path = DATA_PATH):
    csv_path = os.path.join(path, 'HTRU_2.csv')
    return np.loadtxt(csv_path, delimiter=',', dtype=np.float32)

def load_pulsar_arff(path = DATA_PATH):
    arff_path = os.path.join(path, 'HTRU_2.arff')
    return arff.loadarff(arff_path)

In [4]:
pulsars = load_pulsar_csv()

In [5]:
import numpy as np

def split_train_dataset(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(test_ratio * len(data))
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data[train_indices,:], data[test_indices,:]

# Use hash of identifier to decide if instance goes into train or test set

In [6]:
# Save Model
import os

def save_model(model, name, acc=None):
    name += str(model.input.shape[1])
    for layer in model.layers:
        name += "-" + str(layer.output.shape[1])
    
    name += "_" + (("%.2f" % acc) if acc is not None else "")
    path = os.path.join(MODELS_DIR, name + ".h5")
    model.save(path)

In [7]:
train_set, test_set = split_train_dataset(pulsars, 0.2)

In [8]:
X_train, Y_train = train_set[:, :-1], train_set[:, -1]
X_test, Y_test = test_set[:, :-1], test_set[:, -1]

In [15]:
from keras import regularizers

# Create Model
input_dimension = np.size(X_train, axis=1)

def create_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_dimension,), scale=False)) # scaling will be handled by next non-linear layer (relu)
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
# Create and Compile Model
model = create_model()

In [17]:
# Fit the Model
model.fit(X_train, Y_train, epochs=150, batch_size=15, validation_data=[X_test, Y_test])

Train on 14319 samples, validate on 3579 samples
Epoch 1/150
14319/14319 [==============================] - 5s 320us/step - loss: 0.1973 - acc: 0.9312 - val_loss: 0.0951 - val_acc: 0.9715
Epoch 2/150
14319/14319 [==============================] - 4s 305us/step - loss: 0.1122 - acc: 0.9639 - val_loss: 0.0847 - val_acc: 0.9740
Epoch 3/150
14319/14319 [==============================] - 4s 313us/step - loss: 0.1093 - acc: 0.9652 - val_loss: 0.0842 - val_acc: 0.9743
Epoch 4/150
14319/14319 [==============================] - 4s 305us/step - loss: 0.1093 - acc: 0.9654 - val_loss: 0.0846 - val_acc: 0.9743
Epoch 5/150
14319/14319 [==============================] - 5s 323us/step - loss: 0.1037 - acc: 0.9665 - val_loss: 0.0823 - val_acc: 0.9749
Epoch 6/150
14319/14319 [==============================] - 5s 319us/step - loss: 0.1045 - acc: 0.9673 - val_loss: 0.0797 - val_acc: 0.9768
Epoch 7/150
14319/14319 [==============================] - 5s 336us/step - loss: 0.1031 - acc: 0.9664 - val_loss: 0.0

14319/14319 [==============================] - 4s 296us/step - loss: 0.0913 - acc: 0.9702 - val_loss: 0.0715 - val_acc: 0.9793
Epoch 118/150
14319/14319 [==============================] - 4s 295us/step - loss: 0.0931 - acc: 0.9710 - val_loss: 0.0733 - val_acc: 0.9785
Epoch 119/150
14319/14319 [==============================] - 5s 339us/step - loss: 0.0913 - acc: 0.9700 - val_loss: 0.0719 - val_acc: 0.9774
Epoch 120/150
14319/14319 [==============================] - 5s 353us/step - loss: 0.0909 - acc: 0.9716 - val_loss: 0.0725 - val_acc: 0.9779
Epoch 121/150
14319/14319 [==============================] - 5s 351us/step - loss: 0.0922 - acc: 0.9704 - val_loss: 0.0741 - val_acc: 0.9782
Epoch 122/150
14319/14319 [==============================] - 5s 316us/step - loss: 0.0913 - acc: 0.9723 - val_loss: 0.0717 - val_acc: 0.9785
Epoch 123/150
14319/14319 [==============================] - 4s 314us/step - loss: 0.0897 - acc: 0.9711 - val_loss: 0.0721 - val_acc: 0.9790
Epoch 124/150
14319/14319 [

In [18]:
# Evaluate the Model
scores = model.evaluate(X_test, Y_test)

3579/3579 [==============================] - 0s 55us/step


In [19]:
print("Accuracy:", scores[1] * 100)

Accuracy: 97.7647387588


In [20]:
from sklearn.metrics import classification_report

# Classification metrics on test data
predictions = model.predict(X_test)
Y_test_predictions = [int(y + 0.5) for y in predictions]
print(classification_report(Y_test, Y_test_predictions))

             precision    recall  f1-score   support

        0.0       0.98      0.99      0.99      3259
        1.0       0.93      0.81      0.87       320

avg / total       0.98      0.98      0.98      3579



In [21]:
# Classification metrics on all data

predictions = model.predict(pulsars[:, :-1])
Y_test_predictions = [int(y + 0.5) for y in predictions]
print(classification_report(pulsars[:,-1].astype(int), Y_test_predictions))

             precision    recall  f1-score   support

          0       0.98      0.99      0.99     16259
          1       0.94      0.80      0.86      1639

avg / total       0.98      0.98      0.98     17898



In [22]:
save_model(model, 'pulsar_norm', scores[1] * 100)